# Module 09: Resolving Merge Conflicts - Deep Dive

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 90-120 minutes

**Prerequisites**: 
- Module 03: Branching and Merging
- Module 04: Collaboration Workflows
- Basic understanding of Git workflow

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand what merge conflicts are and why they occur
2. Identify different types of merge conflicts
3. Read and interpret conflict markers in files
4. Resolve conflicts manually using text editors
5. Use merge tools to resolve conflicts visually
6. Handle special conflicts (binary files, notebooks, data files)
7. Apply strategies to prevent conflicts
8. Recover from failed merge attempts

---

## 1. Understanding Merge Conflicts

### What is a Merge Conflict?

A **merge conflict** occurs when Git cannot automatically combine changes from different branches because:
- The same line(s) were modified differently in both branches
- A file was modified in one branch and deleted in another
- Binary files (images, PDFs) were changed in both branches

### Why Conflicts Happen

```
Initial State (main branch):
─────────────────────────────
Line 1: Hello World
Line 2: This is a test
Line 3: End of file

Branch A modifies:              Branch B modifies:
─────────────────              ─────────────────
Line 1: Hello World            Line 1: Hello World
Line 2: This is version A      Line 2: This is version B
Line 3: End of file            Line 3: End of file

When merging: Git doesn't know which version of Line 2 to keep!
```

### Important: Conflicts Are Normal!

- Conflicts are a **normal part** of collaborative development
- They indicate areas where human judgment is needed
- Learning to resolve them confidently is a crucial skill
- Most conflicts are straightforward to resolve

---

## 2. Setting Up a Practice Environment

Let's create a safe environment to practice resolving conflicts.

In [ ]:
import os
import subprocess
from pathlib import Path

# Create a practice directory
practice_dir = Path("merge_conflict_practice")
practice_dir.mkdir(exist_ok=True)
os.chdir(practice_dir)

print(f"Created practice directory: {practice_dir.absolute()}")
print("\nWe'll use this directory to safely practice conflict resolution.")

In [ ]:
# Helper function to run git commands
def run_git_command(command, capture_output=True):
    """
    Execute a git command and return the result.
    
    Args:
        command: Git command as a string
        capture_output: Whether to capture and return output
    
    Returns:
        Output of the command if successful, error message if failed
    """
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=capture_output,
            text=True,
            check=False
        )
        
        if result.returncode != 0:
            return f"Error: {result.stderr}"
        
        return result.stdout if capture_output else None
        
    except Exception as e:
        return f"Exception occurred: {str(e)}"

print("Helper function defined successfully!")

---

## 3. Creating Your First Conflict (Intentionally)

Let's deliberately create a conflict to understand how they work.

In [ ]:
# Initialize a new Git repository
!git init

# Configure git for this repository
!git config user.name "Conflict Learner"
!git config user.email "learner@example.com"

print("\n✓ Repository initialized!")

In [ ]:
# Create a simple Python file
initial_content = """# Data Analysis Script

def greet():
    print("Hello from main branch")

def analyze_data():
    print("Analyzing data...")
    return [1, 2, 3, 4, 5]

if __name__ == "__main__":
    greet()
    data = analyze_data()
    print(f"Data: {data}")
"""

# Write the file
with open("analysis.py", "w") as f:
    f.write(initial_content)

print("✓ Created analysis.py")
print("\nFile contents:")
print(initial_content)

In [ ]:
# Make initial commit
!git add analysis.py
!git commit -m "Initial commit: Add analysis script"

print("✓ Made initial commit on main branch")

In [ ]:
# Create a feature branch
!git checkout -b feature-greeting

# Modify the greet function in feature branch
feature_content = """# Data Analysis Script

def greet():
    print("Welcome to the advanced data analyzer!")
    print("Feature branch version")

def analyze_data():
    print("Analyzing data...")
    return [1, 2, 3, 4, 5]

if __name__ == "__main__":
    greet()
    data = analyze_data()
    print(f"Data: {data}")
"""

with open("analysis.py", "w") as f:
    f.write(feature_content)

!git add analysis.py
!git commit -m "Feature: Improve greeting message"

print("✓ Created feature-greeting branch with changes")

In [ ]:
# Switch back to main and make different changes to same lines
!git checkout main

main_content = """# Data Analysis Script

def greet():
    print("Hello from the main branch!")
    print("Production version")

def analyze_data():
    print("Analyzing data...")
    return [1, 2, 3, 4, 5]

if __name__ == "__main__":
    greet()
    data = analyze_data()
    print(f"Data: {data}")
"""

with open("analysis.py", "w") as f:
    f.write(main_content)

!git add analysis.py
!git commit -m "Main: Update greeting message"

print("✓ Modified same lines on main branch")
print("\nNow we have conflicting changes in both branches!")

---

## 4. Triggering the Conflict

Let's try to merge the feature branch into main and see what happens.

In [ ]:
# Attempt to merge - this will create a conflict
merge_result = run_git_command("git merge feature-greeting")

print("Merge attempt output:")
print("=" * 50)
print(merge_result)
print("=" * 50)
print("\n⚠️  CONFLICT DETECTED!")
print("This is expected - we intentionally created conflicting changes.")

In [ ]:
# Check git status to see the conflict
!git status

### Understanding the Status Output

Key indicators of a conflict:
- `You have unmerged paths` - Merge is in progress but blocked
- `both modified: analysis.py` - Both branches changed this file
- Files are listed under "Unmerged paths"

---

## 5. Reading Conflict Markers

Let's examine the conflicted file to understand Git's conflict markers.

In [ ]:
# Read and display the conflicted file
with open("analysis.py", "r") as f:
    conflicted_content = f.read()

print("Conflicted file contents:")
print("=" * 60)
print(conflicted_content)
print("=" * 60)

### Anatomy of Conflict Markers

Git inserts special markers to show the conflicting changes:

```python
<<<<<<< HEAD
# This is YOUR current branch's version (main)
def greet():
    print("Hello from the main branch!")
    print("Production version")
=======
# This is the INCOMING branch's version (feature-greeting)
def greet():
    print("Welcome to the advanced data analyzer!")
    print("Feature branch version")
>>>>>>> feature-greeting
```

**Key Markers**:
- `<<<<<<< HEAD` - Start of current branch's version
- `=======` - Separator between versions
- `>>>>>>> feature-greeting` - End of incoming branch's version

**To resolve**: Remove the markers and choose which code to keep (or combine both).

---

## 6. Resolving the Conflict Manually

Let's resolve this conflict by choosing the best elements from both versions.

In [ ]:
# Strategy: Combine the best of both versions
# We'll use the more descriptive greeting from feature branch
# but keep the "Production" indicator from main

resolved_content = """# Data Analysis Script

def greet():
    print("Welcome to the advanced data analyzer!")
    print("Production version")

def analyze_data():
    print("Analyzing data...")
    return [1, 2, 3, 4, 5]

if __name__ == "__main__":
    greet()
    data = analyze_data()
    print(f"Data: {data}")
"""

# Write the resolved version
with open("analysis.py", "w") as f:
    f.write(resolved_content)

print("✓ Conflict resolved by combining both versions")
print("\nResolved content:")
print(resolved_content)

In [ ]:
# Mark the conflict as resolved
!git add analysis.py

print("✓ Marked conflict as resolved with 'git add'")
print("\nCheck status:")
!git status

In [ ]:
# Complete the merge with a commit
!git commit -m "Merge: Resolve conflict by combining greeting messages"

print("\n✓ Merge completed successfully!")

### Steps to Resolve Any Conflict

1. **Identify** conflicted files: `git status`
2. **Open** each conflicted file
3. **Locate** conflict markers (`<<<<<<<`, `=======`, `>>>>>>>`)
4. **Decide** which changes to keep:
   - Keep current branch's version
   - Keep incoming branch's version
   - Combine both versions
   - Write entirely new code
5. **Remove** all conflict markers
6. **Test** that the code still works
7. **Stage** resolved files: `git add <file>`
8. **Commit** the merge: `git commit`

---

## 7. Types of Conflicts

Let's explore different types of conflicts you'll encounter.

### Type 1: Content Conflicts (Most Common)

When the same lines are modified in both branches.

**Example**: Two developers editing the same function

**Resolution Strategy**:
- Read both versions carefully
- Understand the intent behind each change
- Combine or choose the better approach
- Test thoroughly after resolution

---

### Type 2: Modify/Delete Conflicts

When one branch modifies a file that another branch deleted.

In [ ]:
# Create a new file
with open("old_module.py", "w") as f:
    f.write("# Old deprecated module\nprint('This is old')")

!git add old_module.py
!git commit -m "Add old module"

# Create branch that modifies it
!git checkout -b update-module
with open("old_module.py", "w") as f:
    f.write("# Updated module\nprint('This is updated')")
!git add old_module.py
!git commit -m "Update old module"

# Switch back and delete it
!git checkout main
!git rm old_module.py
!git commit -m "Remove deprecated module"

print("✓ Setup complete for modify/delete conflict")

In [ ]:
# Try to merge - this creates a modify/delete conflict
merge_result = run_git_command("git merge update-module")
print(merge_result)

!git status

**Resolution Options**:

```bash
# Option 1: Keep the file (and the modifications)
git add old_module.py
git commit

# Option 2: Confirm the deletion
git rm old_module.py
git commit
```

**Decision factors**:
- Is the file truly obsolete?
- Are the modifications important?
- Can functionality be moved to a new file?

In [ ]:
# Let's confirm the deletion
!git rm old_module.py
!git commit -m "Merge: Confirm deletion of deprecated module"

print("✓ Resolved modify/delete conflict by confirming deletion")

---

## 8. Aborting a Merge

Sometimes you need to stop a merge and start over.

In [ ]:
# Create another conflict scenario
with open("config.py", "w") as f:
    f.write("DATABASE_URL = 'localhost'")

!git add config.py
!git commit -m "Add config file"

# Create conflicting branches
!git checkout -b config-dev
with open("config.py", "w") as f:
    f.write("DATABASE_URL = 'dev.example.com'")
!git add config.py
!git commit -m "Update to dev database"

!git checkout main
with open("config.py", "w") as f:
    f.write("DATABASE_URL = 'prod.example.com'")
!git add config.py
!git commit -m "Update to prod database"

print("✓ Created another conflict scenario")

In [ ]:
# Start merge (will conflict)
result = run_git_command("git merge config-dev")
print(result)
print("\n⚠️  Conflict created")

In [ ]:
# Abort the merge - return to state before merge
!git merge --abort

print("✓ Merge aborted successfully!")
print("\nCurrent status:")
!git status

### When to Abort a Merge

Use `git merge --abort` when:
- Conflicts are more complex than expected
- You need to discuss resolution strategy with team
- You accidentally merged the wrong branch
- You need to pull latest changes first
- You want to try a different approach

**Important**: `--abort` only works during an active merge. Once you commit, use `git reset` instead.

---

## 9. Preventing Conflicts

While conflicts are normal, you can minimize them.

### Best Practices to Avoid Conflicts

1. **Pull frequently**: Keep your branch up to date
   ```bash
   git pull origin main
   ```

2. **Communicate with team**: Coordinate who's working on what
   - Use issue tracking systems
   - Assign code owners to files
   - Have daily standups

3. **Make small, focused commits**: Easier to merge
   - One feature per branch
   - Commit logical units of work
   - Merge branches regularly

4. **Organize code well**: Reduce overlap
   - Modular design
   - Separate files for different features
   - Clear function boundaries

5. **Merge main into feature branches**: Not the other way around
   ```bash
   # On feature branch
   git merge main
   # Resolve conflicts here, not on main
   ```

6. **Use code formatting tools**: Consistent style
   - Black for Python
   - Prettier for JavaScript
   - Configure in pre-commit hooks

---

## 10. Data Science Specific Conflicts

Special considerations for data science workflows.

### Jupyter Notebook Conflicts

Notebook JSON structure makes conflicts messy:

```json
<<<<<<< HEAD
   "execution_count": 5,
   "outputs": [{"text": "Result: 42"}]
=======
   "execution_count": 3,
   "outputs": [{"text": "Result: 42"}]
>>>>>>> feature
```

**Solutions**:

1. **Strip outputs before committing**: Use `nbstripout`
   ```bash
   pip install nbstripout
   nbstripout --install
   ```

2. **Use nbdime**: Notebook-aware diff/merge tool
   ```bash
   pip install nbdime
   nbdime config-git --enable
   ```

3. **Convert to Python scripts**: For version control
   ```bash
   jupyter nbconvert --to script notebook.ipynb
   ```

---

### Data File Conflicts

**CSV Files**: Can be merged but tedious
- **Solution**: Don't version processed data
- Version the scripts that generate data instead
- Use data versioning tools (DVC)

**Binary Files**: Cannot be merged automatically
- Images, PDFs, pickled models
- **Solution**: Choose one version or rename both
```bash
# Keep current version
git checkout --ours model.pkl

# Keep incoming version  
git checkout --theirs model.pkl
```

---

## 11. Using Merge Tools

Visual merge tools make complex conflicts easier to resolve.

### Popular Merge Tools

1. **VS Code**: Built-in merge editor
   ```bash
   git config --global merge.tool vscode
   git config --global mergetool.vscode.cmd 'code --wait $MERGED'
   ```

2. **KDiff3**: Powerful 3-way merge
   ```bash
   git config --global merge.tool kdiff3
   ```

3. **Meld**: User-friendly visual diff
   ```bash
   git config --global merge.tool meld
   ```

4. **P4Merge**: Free from Perforce
   ```bash
   git config --global merge.tool p4merge
   ```

### Using a Merge Tool

```bash
# During a conflict
git mergetool

# This opens your configured tool
# Typically shows three panels:
# - Left: Current branch (HEAD)
# - Right: Incoming branch
# - Bottom: Result (what you're creating)
```

---

## 12. Advanced Conflict Resolution

Techniques for complex scenarios.

### Taking One Side Entirely

When you know you want one version completely:

```bash
# Keep all changes from current branch (main)
git checkout --ours file.py
git add file.py

# Keep all changes from incoming branch
git checkout --theirs file.py
git add file.py
```

**Use cases**:
- Configuration files (keep prod vs dev)
- Binary files that can't be merged
- Complete rewrites where one version is clearly better

---

### Finding the Common Ancestor

Sometimes you need to see the original version:

```bash
# During a conflict, view the base version
git show :1:file.py

# View current branch's version
git show :2:file.py

# View incoming branch's version
git show :3:file.py
```

This helps understand what changed in each branch.

---

## 13. Exercise 1: Resolve a Simple Conflict

**Task**: Create and resolve a conflict in a data analysis script.

**Steps**:
1. Create a new repository
2. Add a script that loads and processes data
3. Create two branches that modify the same function differently
4. Merge and resolve the conflict
5. Verify the merged code works

**Starter code provided below**:

In [ ]:
# Exercise 1: Your solution here

# Step 1: Create exercise directory
exercise_dir = Path("../exercise1_conflicts")
exercise_dir.mkdir(exist_ok=True)

# Your code here
# Hint: Follow the pattern from Section 3

print("TODO: Complete this exercise")

---

## 14. Exercise 2: Handle a Modify/Delete Conflict

**Task**: Create and resolve a modify/delete conflict.

**Scenario**: One branch updates a legacy data processing function, while another branch removes it in favor of a new approach.

**Decision**: Determine whether to keep the updated version or confirm the deletion.

In [ ]:
# Exercise 2: Your solution here

# Create a scenario where:
# - Branch A: Refactors an old function
# - Branch B: Deletes the old function and adds a new one

# Your code here

print("TODO: Complete this exercise")

---

## 15. Exercise 3: Conflict Prevention Strategy

**Task**: Design a workflow that minimizes conflicts for a data science team.

**Requirements**:
1. 3 team members working on same project
2. Each person working on different features
3. Regular integration of changes
4. Minimal merge conflicts

**Deliverable**: Write a markdown cell below describing your strategy.

### Your Strategy Here

TODO: Describe your conflict prevention strategy

Consider:
- Branch naming conventions
- Merge frequency
- Code organization
- Communication practices
- Tools and automation

---

## 16. Summary

### Key Concepts Learned

1. **Conflicts are normal**: They're an expected part of collaboration
2. **Conflict markers**: Understand `<<<<<<<`, `=======`, `>>>>>>>`
3. **Resolution process**: Identify → Edit → Stage → Commit
4. **Types of conflicts**: Content, modify/delete, binary files
5. **Aborting merges**: Use `git merge --abort` when needed
6. **Prevention strategies**: Pull frequently, communicate, organize code
7. **Data science specifics**: Handle notebooks and data files carefully
8. **Tools**: Visual merge tools make complex conflicts easier

### Essential Commands

```bash
# During a conflict
git status                    # See conflicted files
git diff                      # View conflict details
git add <file>                # Mark as resolved
git commit                    # Complete the merge
git merge --abort             # Cancel the merge

# Advanced resolution
git checkout --ours <file>    # Take current branch's version
git checkout --theirs <file>  # Take incoming branch's version
git mergetool                 # Open visual merge tool
```

### Best Practices

✅ **DO**:
- Read both versions carefully before resolving
- Test code after resolving conflicts
- Write clear merge commit messages
- Use visual tools for complex conflicts
- Strip notebook outputs before committing
- Communicate with team about conflicts

❌ **DON'T**:
- Randomly delete conflict markers
- Force push after resolving conflicts
- Ignore the meaning of conflicting changes
- Resolve conflicts without testing
- Version large binary data files
- Panic when conflicts occur

---

## 17. What's Next?

Now that you can confidently handle merge conflicts, you're ready for:

**Module 10: Git Best Practices**
- Writing effective commit messages
- Using .gitignore properly
- Workflow strategies for teams
- Code review best practices

### Additional Resources

- [Git Documentation - Merge Conflicts](https://git-scm.com/docs/git-merge#_how_conflicts_are_presented)
- [Atlassian Git Tutorial - Merge Conflicts](https://www.atlassian.com/git/tutorials/using-branches/merge-conflicts)
- [nbdime - Jupyter Notebook Diff/Merge](https://nbdime.readthedocs.io/)
- [Visual Studio Code - Merge Conflicts](https://code.visualstudio.com/docs/editor/versioncontrol#_merge-conflicts)

### Practice More

The best way to get comfortable with conflicts is to practice:
1. Create intentional conflicts in safe environments
2. Try different resolution strategies
3. Use various merge tools
4. Collaborate on real projects

---

**Remember**: Every developer deals with merge conflicts. The difference between juniors and seniors is confidence in resolving them!

**Great job completing this module!** 🎉